## FASTQ Mutation Search Code

### Generate CSV

In [1]:
#!/usr/bin/env python3

import gzip
from collections import defaultdict
import csv
import os
import numpy as np
import glob

def read_fastq(file_path: str):
    """Read FASTQ file and yield sequences."""
    open_func = gzip.open if file_path.endswith('.gz') else open
    mode = 'rt' if file_path.endswith('.gz') else 'r'
    
    with open_func(file_path, mode) as f:
        while True:
            header = f.readline().strip()
            if not header:
                break
            sequence = f.readline().strip()
            _ = f.readline()  
            _ = f.readline() 
            yield sequence

def count_patterns(sequence: str, pattern: str) -> int:
    return sequence.count(pattern)

def load_age_data():
    """Load age data from greider_methods_table_s2.csv."""
    age_data = {}
    with open('greider_methods_table_s2.csv', 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            fastq_name = row['fastq file name'].replace('_', '.')
            age_data[fastq_name] = row['Age (Years)']
    return age_data

def load_length_data():
    """Load length data from greider_methods_table_s2.csv."""
    length_data = {}
    with open('greider_methods_table_s2.csv', 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            fastq_name = row['fastq file name'].replace('_', '.')
            length_data[fastq_name] = row['Mean Telomere Length (bps)']
    return length_data

def get_fastq_files(directory: str):
    """Get all FASTQ files in the given directory."""
    # Look for both .fastq and .fastq.gz files
    fastq_files = glob.glob(os.path.join(directory, "*.fastq"))
    fastq_files.extend(glob.glob(os.path.join(directory, "*.fastq.gz")))
    return sorted(fastq_files)  # Sort for consistent ordering

def generate_csv(data_dir: str):
    fastq_files = get_fastq_files(data_dir)
    
    if not fastq_files:
        print(f"No FASTQ files found in {data_dir} directory")
        return
    
    # Load age data
    age_data = load_age_data()
    length_data = load_length_data()
    
    patterns = {
        'c_strand': "CCCTAACCCTAA",
        'g_strand': "GGGTTAGGGTTA",
        'g_strand_mutations': {
            'G>A_g1': "GGGTTAAGGTTA",
            'G>A_g2': "GGGTTAGAGTTA",
            'G>A_g3': "GGGTTAGGATTA",
            'G>C_g1': "GGGTTACGGTTA",
            'G>C_g2': "GGGTTAGCGTTA",
            'G>C_g3': "GGGTTAGGCTTA",
            'G>T_g1': "GGGTTATGGTTA",
            'G>T_g2': "GGGTTAGTGTTA",
            'G>T_g3': "GGGTTAGGTTTA",
            'T>A_t1': "GGGTTAGGGATA",
            'T>A_t2': "GGGTTAGGGTAA",
            'T>C_t1': "GGGTCAGGGTTA",
            'T>C_t2': "GGGTTACGGTTA",
            'T>G_t1': "GGGTGAGGGTTA",
            'T>G_t2': "GGGTTAGGGGTA",
            'A>T_a1': "GGGTTAGGGTTT",
            'A>G_a1': "GGGTTAGGGTTG",
            'A>C_a1': "GGGTTAGGGTTC",
        },
        'c_strand_mutations': {
            'C>A_c1': "CCCTAAACCTAA",
            'C>A_c2': "CCCTAACACTAA",
            'C>A_c3': "CCCTAACCATAA",
            'C>G_c1': "CCCTAAGCCTAA",
            'C>G_c2': "CCCTAACGCTAA",
            'C>G_c3': "CCCTAACCGTAA",
            'C>T_c1': "CCCTAATCCTAA",
            'C>T_c2': "CCCTAACTCTAA",
            'C>T_c3': "CCCTAACCTTAA",
            'T>A_t1': "CCCTAACCCAAA",
            'T>C_t1': "CCCTAACCCCAA",
            'T>G_t1': "CCCTAACCCGAA",
            'A>T_a1': "CCCTAACCCTTA",
            'A>T_a2': "CCCTAACCCTAT",
            'A>G_a1': "CCCTAACCCTGA",
            'A>G_a2': "CCCTAACCCTAG",
            'A>C_a1': "CCCTAACCCTCA",
            'A>C_a2': "CCCTAACCCTAC",
        },
    }
    
    with open('telomere_analysis.csv', 'w', newline='') as csvfile:
        fieldnames = [
            'FileName', 'Age', 'Telomere_Length',
            'c_strand', 'g_strand',
        ]
        mutation_keys = []
        for group in ['g_strand_mutations', 'c_strand_mutations']:
            for subkey in patterns[group].keys():
                mutation_keys.append(f"{group}_{subkey}")
        fieldnames.extend(mutation_keys)

        # Add normalized rate fields for each mutation key
        fieldnames.extend([f"{k}_per_1k" for k in mutation_keys])

        # --- Add general mutation per_1k columns ---
        general_mutation_map = {
            'g_strand': {
                'G>A': ['G>A_g1', 'G>A_g2', 'G>A_g3'],
                'G>C': ['G>C_g1', 'G>C_g2', 'G>C_g3'],
                'G>T': ['G>T_g1', 'G>T_g2', 'G>T_g3'],
                'T>A': ['T>A_t1', 'T>A_t2'],
                'T>C': ['T>C_t1', 'T>C_t2'],
                'T>G': ['T>G_t1', 'T>G_t2'],
                'A>T': ['A>T_a1'],
                'A>G': ['A>G_a1'],
                'A>C': ['A>C_a1'],
            },
            'c_strand': {
                'C>A': ['C>A_c1', 'C>A_c2', 'C>A_c3'],
                'C>G': ['C>G_c1', 'C>G_c2', 'C>G_c3'],
                'C>T': ['C>T_c1', 'C>T_c2', 'C>T_c3'],
                'T>A': ['T>A_t1'],
                'T>C': ['T>C_t1'],
                'T>G': ['T>G_t1'],
                'A>T': ['A>T_a1', 'A>T_a2'],
                'A>G': ['A>G_a1', 'A>G_a2'],
                'A>C': ['A>C_a1', 'A>C_a2'],
            }
        }
        general_mutation_headers = []
        for strand, mutmap in general_mutation_map.items():
            for mut in mutmap:
                general_mutation_headers.append(f"{strand}_{mut}_per_1k")
        fieldnames.extend(general_mutation_headers)

        # Add new engineered features to the CSV header
        fieldnames.extend([
            'composite_transition_per_1k',
            'composite_transversion_per_1k',
            'mutation_ratio_CtoA_CtoT',
            'mutation_ratio_GtoA_GtoT',
            'g_strand_mutations_sum_per_1k',
            'c_strand_mutations_sum_per_1k',
            'log_telomere_length',
            'telomere_length_bin',
            'telomere_transition_interaction'
        ])

        # Add summed per-1k columns for each mutation type per strand
        summed_per_1k_headers = []
        for strand, mutmap in general_mutation_map.items():
            for mut, subtypes in mutmap.items():
                summed_per_1k_headers.append(f"{strand}_{mut}_sum_per_1k")
        fieldnames.extend(summed_per_1k_headers)

        # Add total mutations field at the end
        fieldnames.append('total_mutations_over_total_g_strand_2xrepeats_per_1k')
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for file_path in fastq_files:
            counts = defaultdict(int)
            
            for sequence in read_fastq(file_path):
                # Count c-strand in forward direction only
                counts['c_strand'] += count_patterns(sequence, patterns['c_strand'])
                # Count g-strand in forward direction only
                counts['g_strand'] += count_patterns(sequence, patterns['g_strand'])
                # Count all mutation sub-patterns
                for group in ['g_strand_mutations', 'c_strand_mutations']:
                    for subkey, subpattern in patterns[group].items():
                        counts[f"{group}_{subkey}"] += count_patterns(sequence, subpattern)
            
            filename = os.path.basename(file_path)
            filename_base = filename.replace('.fastq', '').replace('.gz', '')
            age = age_data.get(filename_base, '')
            length = length_data.get(filename_base, '')
            g_strand_total = counts['g_strand']
            c_strand_total = counts['c_strand']

            # Calculate per 1k rates for each mutation type
            def per_1k(val, total):
                return (val / total) * 1000 if total > 0 else 0
            row = {
                'FileName': filename,
                'Age': age,
                'Telomere_Length': length,
                'c_strand': c_strand_total,
                'g_strand': g_strand_total,
            }

            for k in mutation_keys:
                row[k] = counts.get(k, 0)
                if k.startswith('g_strand_mutations'):
                    norm_total = g_strand_total
                elif k.startswith('c_strand_mutations'):
                    norm_total = c_strand_total
                else:
                    norm_total = g_strand_total  # fallback, should not occur
                row[f"{k}_per_1k"] = per_1k(counts.get(k, 0), norm_total)

            # --- General mutation per_1k columns ---
            for strand, mutmap in general_mutation_map.items():
                strand_total = g_strand_total if strand == 'g_strand' else c_strand_total
                for mut, subtypes in mutmap.items():
                    total = sum(counts.get(f"{strand}_mutations_{subtype}", 0) for subtype in subtypes)
                    row[f"{strand}_{mut}_per_1k"] = per_1k(total, strand_total)

            # --- Summed per-1k columns for each mutation type per strand ---
            for strand, mutmap in general_mutation_map.items():
                for mut, subtypes in mutmap.items():
                    per_1k_sum = sum(row.get(f"{strand}_mutations_{subtype}_per_1k", 0) for subtype in subtypes)
                    row[f"{strand}_{mut}_sum_per_1k"] = per_1k_sum
            
            # Total mutations (sum all mutation counts)
            total_mutations = sum(counts[k] for k in mutation_keys)
            row['total_mutations_over_total_g_strand_2xrepeats_per_1k'] = per_1k(total_mutations, g_strand_total)

            # --- New engineered features ---
           

            # Composite scores (per_1k only)
            transition_keys = [
                'g_strand_mutations_G>A_g1_per_1k', 'g_strand_mutations_G>A_g2_per_1k', 'g_strand_mutations_G>A_g3_per_1k',
                'c_strand_mutations_C>T_c1_per_1k', 'c_strand_mutations_C>T_c2_per_1k', 'c_strand_mutations_C>T_c3_per_1k',
                'g_strand_mutations_A>G_a1_per_1k', 'c_strand_mutations_A>G_a1_per_1k', 'c_strand_mutations_A>G_a2_per_1k',
                'g_strand_mutations_T>C_t1_per_1k', 'g_strand_mutations_T>C_t2_per_1k'
            ]
            transversion_keys = [
                'g_strand_mutations_G>C_g1_per_1k', 'g_strand_mutations_G>C_g2_per_1k', 'g_strand_mutations_G>C_g3_per_1k',
                'g_strand_mutations_G>T_g1_per_1k', 'g_strand_mutations_G>T_g2_per_1k', 'g_strand_mutations_G>T_g3_per_1k',
                'g_strand_mutations_T>A_t1_per_1k', 'g_strand_mutations_T>A_t2_per_1k',
                'g_strand_mutations_T>G_t1_per_1k', 'g_strand_mutations_T>G_t2_per_1k',
                'g_strand_mutations_A>T_a1_per_1k', 'g_strand_mutations_A>C_a1_per_1k',
                'c_strand_mutations_C>A_c1_per_1k', 'c_strand_mutations_C>A_c2_per_1k', 'c_strand_mutations_C>A_c3_per_1k',
                'c_strand_mutations_C>G_c1_per_1k', 'c_strand_mutations_C>G_c2_per_1k', 'c_strand_mutations_C>G_c3_per_1k',
                'c_strand_mutations_T>A_t1_per_1k', 'c_strand_mutations_T>G_t1_per_1k',
                'c_strand_mutations_A>T_a1_per_1k', 'c_strand_mutations_A>T_a2_per_1k',
                'c_strand_mutations_A>C_a1_per_1k', 'c_strand_mutations_A>C_a2_per_1k'
            ]

            row['composite_transition_per_1k'] = sum(row.get(k, 0) for k in transition_keys)
            row['composite_transversion_per_1k'] = sum(row.get(k, 0) for k in transversion_keys)

            # Mutation ratios
            def safe_ratio(a, b):
                return a / b if b != 0 else 0

            row['mutation_ratio_CtoA_CtoT'] = safe_ratio(
                sum(row.get(k, 0) for k in ['c_strand_mutations_C>A_c1_per_1k', 'c_strand_mutations_C>A_c2_per_1k', 'c_strand_mutations_C>A_c3_per_1k']),
                sum(row.get(k, 0) for k in ['c_strand_mutations_C>T_c1_per_1k', 'c_strand_mutations_C>T_c2_per_1k', 'c_strand_mutations_C>T_c3_per_1k']) + 1e-6
            )
            row['mutation_ratio_GtoA_GtoT'] = safe_ratio(
                sum(row.get(k, 0) for k in ['g_strand_mutations_G>A_g1_per_1k', 'g_strand_mutations_G>A_g2_per_1k', 'g_strand_mutations_G>A_g3_per_1k']),
                sum(row.get(k, 0) for k in ['g_strand_mutations_G>T_g1_per_1k', 'g_strand_mutations_G>T_g2_per_1k', 'g_strand_mutations_G>T_g3_per_1k']) + 1e-6
            )

            # Composite per strand
            row['g_strand_mutations_sum_per_1k'] = sum(row.get(k, 0) for k in row if k.startswith('g_strand_mutations') and k.endswith('_per_1k'))
            row['c_strand_mutations_sum_per_1k'] = sum(row.get(k, 0) for k in row if k.startswith('c_strand_mutations') and k.endswith('_per_1k'))

            # Log-transformed telomere length
            try:
                row['log_telomere_length'] = np.log1p(float(row['Telomere_Length'])) if row['Telomere_Length'] else 0
            except Exception:
                row['log_telomere_length'] = 0

            # Telomere length bin
            try:
                length_val = float(row['Telomere_Length'])
                if length_val < 5000:
                    row['telomere_length_bin'] = 'short'
                elif length_val < 8000:
                    row['telomere_length_bin'] = 'medium'
                else:
                    row['telomere_length_bin'] = 'long'
            except Exception:
                row['telomere_length_bin'] = 'unknown'

            # Interaction term (telomere length × composite_transition_per_1k)
            try:
                row['telomere_transition_interaction'] = float(row['Telomere_Length']) * row['composite_transition_per_1k']
            except Exception:
                row['telomere_transition_interaction'] = 0

            writer.writerow(row)
            
            print(f"\nProcessing {filename}:")
            print(f"Age: {age}")
            print(f"Telomere Length: {length}")
            print(f"2x cstrand total: {c_strand_total}")
            print(f"2x g strand total: {g_strand_total}")
            if counts['g_strand'] == 0 and counts['c_strand'] == 0:
                print(f"Example sequence from {filename}: {sequence}")

# if __name__ == "__main__":
#     generate_csv()


### Generate Histogram

In [2]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def plot_histograms(data, output_path):
    # Set up the plotting style
    sns.set_style("whitegrid")
    sns.set_palette("husl")
    
    fig, axes = plt.subplots(2, 2, figsize=(20, 12))
    fig.suptitle('Mutation Rates by Age Groups (10-year bins)', fontsize=16, fontweight='bold')
    
    variables = [
        'total_mutations_over_total_g_strand_2xrepeats_per_1k',
        'g_strand_mutations_G>T_g1_per_1k',  # pos1 mutation rate
        'g_strand_mutations_G>T_g2_per_1k',  # pos2 mutation rate  
        'g_strand_mutations_G>T_g3_per_1k'   # pos3 mutation rate
    ]
    
    titles = [
        'Total Mutations per 1000bp',
        'G > T at Position 1 Mutation Rate per 1000bp',
        'G > T at Position 2 Mutation Rate per 1000bp',
        'G > T at Position 3 Mutation Rate per 1000bp'
    ]
    
    for i, (var, title) in enumerate(zip(variables, titles)):
        row = i // 2
        col = i % 2
        ax = axes[row, col]
        
        # Remove rows with missing Age values
        plot_data = data.dropna(subset=['Age', var])
        
        if len(plot_data) > 0:
            # Create age bins of 10 years
            age_bins = np.arange(0, plot_data['Age'].max() + 10, 10)
            age_labels = [f'{int(bin_start)}-{int(bin_start+9)}' for bin_start in age_bins[:-1]]
            
            # Add age group column
            plot_data['Age_Group'] = pd.cut(plot_data['Age'], bins=age_bins, labels=age_labels, include_lowest=True)
            
            # Create box plot with age groups on x-axis and mutation rates on y-axis
            sns.boxplot(data=plot_data, x='Age_Group', y=var, ax=ax)
            
            ax.set_title(title, fontweight='bold')
            ax.set_xlabel('Age Group (years)')
            ax.set_ylabel('Mutations per 1000bp')
            
            # Rotate x-axis labels for better readability
            plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
            
        else:
            ax.text(0.5, 0.5, 'No data available', ha='center', va='center', 
                   transform=ax.transAxes, fontsize=12)
            ax.set_title(title, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()

def plot_histograms_main():
    data = pd.read_csv("telomere_analysis.csv")
    # Plot the histograms
    plot_histograms(data, "histogram.png")
    print("Histogram plot saved as 'histogram.png'")

# if __name__ == "__main__":
#     plot_histograms_main()


### Plotting

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import seaborn as sns

import numbers

def plot_mutational_signature_row(row, mutation_types, mutation_columns, output_path):
    # Set seaborn style for better-looking plots
    sns.set_style("whitegrid")
    sns.set_palette("husl")
    
    # Aggregate counts for each mutation type, position, and strand context for a single row
    bar_heights = []
    bar_colors = []
    bar_labels = []
    # Get all columns for total calculation
    all_columns = []
    for mut_type, contexts in mutation_columns.items():
        for context, cols in contexts.items():
            all_columns.extend(cols)
    total_mutations = row[all_columns].sum()
    for mut_label, color in mutation_types:
        contexts = mutation_columns[mut_label]
        for context_name, cols in contexts.items():
            for i, col in enumerate(cols):
                value = row[col]
                percentage = (value / total_mutations) * 100 if total_mutations > 0 else 0
                bar_heights.append(percentage)
                bar_colors.append(color)
                bar_labels.append(f"{mut_label} {context_name} pos{i+1}")
    
    x = np.arange(len(bar_heights))
    
    # Create figure with seaborn styling
    fig, ax = plt.subplots(figsize=(16, 10))
    
    # Create the bar plot with seaborn styling
    bars = sns.barplot(x=x, y=bar_heights, palette=bar_colors, ax=ax, edgecolor='black', linewidth=0.5)
    
    # Customize the plot
    for i, label in enumerate(bar_labels):
        ax.text(i, -max(bar_heights)*0.02, label, ha='center', va='center', 
                color='black', fontsize=9, fontweight='normal', rotation=45)
    
    ax.set_xticks([])
    ax.set_yticks(np.linspace(0, max(bar_heights), 6))
    ax.set_xlim(-0.5, len(bar_heights) - 0.5)
    ax.set_ylim(-max(bar_heights)*0.1, max(bar_heights) + max(bar_heights)*0.2)
    
    # Remove spines for cleaner look
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    # Add grid with seaborn styling
    ax.yaxis.grid(True, linestyle='--', alpha=0.3)
    ax.set_axisbelow(True)
    
    # Set labels and title with improved styling
    ax.set_ylabel('Percentage of Single Base Modifications', fontsize=14, fontweight='bold')
    
    # Title with age and filename
    age = row['Age'] if 'Age' in row else 'N/A'
    filename = row['FileName'] if 'FileName' in row else 'sample'
    title = f"Mutational Signatures by Position and Strand Context\nFile: {filename} | Age: {age} years"
    ax.set_title(title, fontsize=18, fontweight='bold', pad=30)
    
    # Improve layout
    plt.tight_layout(rect=[0, 0.15, 1, 0.95])
    
    # Save with high quality
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()

def plot_mutational_signatures(csv_path):
    # Set global seaborn style
    sns.set_theme(style="whitegrid", font_scale=1.1)
    
    df = pd.read_csv(csv_path)
    mutation_types = [
        ('C>A', 'blue'),
        ('C>G', 'black'),
        ('C>T', 'red'),
        ('G>A', 'gray'),
        ('G>C', 'green'),
        ('G>T', 'pink'),
    ]
    mutation_columns = {
        'C>A': {
            'C-strand': ['c_strand_mutations_C>A_c1', 'c_strand_mutations_C>A_c2', 'c_strand_mutations_C>A_c3'],
            'G-strand': ['g_strand_mutations_G>T_g1', 'g_strand_mutations_G>T_g2', 'g_strand_mutations_G>T_g3']
        },
        'C>G': {
            'C-strand': ['c_strand_mutations_C>G_c1', 'c_strand_mutations_C>G_c2', 'c_strand_mutations_C>G_c3'],
            'G-strand': ['g_strand_mutations_G>C_g1', 'g_strand_mutations_G>C_g2', 'g_strand_mutations_G>C_g3']
        },
        'C>T': {
            'C-strand': ['c_strand_mutations_C>T_c1', 'c_strand_mutations_C>T_c2', 'c_strand_mutations_C>T_c3'],
            'G-strand': ['g_strand_mutations_G>A_g1', 'g_strand_mutations_G>A_g2', 'g_strand_mutations_G>A_g3']
        },
        'G>A': {
            'G-strand': ['g_strand_mutations_G>A_g1', 'g_strand_mutations_G>A_g2', 'g_strand_mutations_G>A_g3'],
            'C-strand': ['c_strand_mutations_C>G_c1', 'c_strand_mutations_C>G_c2', 'c_strand_mutations_C>G_c3']
        },
        'G>C': {
            'G-strand': ['g_strand_mutations_G>C_g1', 'g_strand_mutations_G>C_g2', 'g_strand_mutations_G>C_g3'],
            'C-strand': ['c_strand_mutations_C>T_c1', 'c_strand_mutations_C>T_c2', 'c_strand_mutations_C>T_c3']
        },
        'G>T': {
            'G-strand': ['g_strand_mutations_G>T_g1', 'g_strand_mutations_G>T_g2', 'g_strand_mutations_G>T_g3'],
            'C-strand': ['c_strand_mutations_C>A_c1', 'c_strand_mutations_C>A_c2', 'c_strand_mutations_C>A_c3']
        }
    }
    # Ensure plots directory exists
    os.makedirs('plots', exist_ok=True)
    for idx, row in df.iterrows():
        filename = str(row['FileName']) if 'FileName' in row else f'sample_{idx}'
        # Remove file extension if present
        filename_base = os.path.splitext(filename)[0]
        output_path = os.path.join('plots', f'{filename_base}.png')
        plot_mutational_signature_row(row, mutation_types, mutation_columns, output_path)

def plot_spearman_with_age(csv_path):
    """
    For each numeric column in the CSV (except 'Age'), plot a scatter plot with Age on the x-axis and the column on the y-axis,
    calculate and display the Spearman correlation, and save each plot in 'spearman's plots' directory.
    Also outputs a CSV table with the r and p values for each column.
    """
    import scipy.stats as stats
    # Set seaborn style for consistency
    sns.set_theme(style="whitegrid", font_scale=1.1)
    # Read data
    df = pd.read_csv(csv_path)
    # Ensure output directory exists
    output_dir = "spearman's plots"
    os.makedirs(output_dir, exist_ok=True)
    # Only keep columns with numeric data and drop rows with missing Age
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if 'Age' not in numeric_cols:
        print("No 'Age' column found in the data.")
        return
    numeric_cols = [col for col in numeric_cols if col != 'Age']
    # Drop rows with missing Age
    df = df.dropna(subset=['Age'])
    spearman_results = []
    for col in numeric_cols:
        # Drop rows with missing values in the current column
        sub_df = df.dropna(subset=[col])
        if sub_df.shape[0] < 2:
            continue  # Not enough data to plot
        x = sub_df['Age']
        y = sub_df[col]
        
        # Calculate Spearman correlation
        corr, pval = stats.spearmanr(x, y)
        spearman_results.append({'Column': col, 'Spearman_r': corr, 'p_value': pval})
        # Plot using seaborn
        plt.figure(figsize=(8, 6))
        ax = sns.scatterplot(x=x, y=y)
        # Add seaborn regression (trendline) with no confidence interval
        if len(x) > 1:
            sns.regplot(x=x, y=y, scatter=False, ci=None, line_kws={'color': 'red', 'linestyle': '--'}, ax=ax)
        ax.set_xlabel('Age (years)', fontsize=12)
        ax.set_ylabel(col, fontsize=12)
        ax.set_title(f"Spearman's ρ = {corr:.2f} (p={pval:.2g})\n{col} vs Age", fontsize=14)
        plt.tight_layout()
        # Save plot
        safe_col = col.replace('/', '_').replace(' ', '_').replace('>', 'to').replace('<', 'lt').replace(':', '_')
        output_path = os.path.join(output_dir, f"{safe_col}_vs_Age.png")
        plt.savefig(output_path, dpi=200)
        plt.close()
    # Output CSV table of results
    results_df = pd.DataFrame(spearman_results)
    results_csv_path = os.path.join(output_dir, "spearman_results.csv")
    results_df.to_csv(results_csv_path, index=False)

# --- Composite Score Plotting ---
def plot_composite_score(csv_path, target_col='Age'):
    """
    Calculate a composite score from selected columns, plot it against the target column,
    and display the Spearman correlation.
    """
    import scipy.stats as stats
    # Columns to combine
    top_features = [
        'total_mutations_over_total_g_strand_2xrepeats_per_1k',
        'g_strand_mutations_A>C_a1_per_1k',
        'g_strand_mutations_T>G_t2_per_1k',
        'g_strand_mutations_T>G_t1_per_1k',
        'g_strand_mutations_G>A_g3_per_1k'
    ]
    df = pd.read_csv(csv_path)
    # Drop rows with missing target
    df = df.dropna(subset=[target_col])
    # Calculate composite score
    df['composite_score'] = df[top_features].mean(axis=1)
    # Calculate Spearman correlation
    r, p = stats.spearmanr(df['composite_score'], df[target_col])
    # Plot
    plt.figure(figsize=(8, 6))
    ax = sns.scatterplot(x=df[target_col], y=df['composite_score'])
    sns.regplot(x=df[target_col], y=df['composite_score'], scatter=False, ci=None, line_kws={'color': 'red', 'linestyle': '--'}, ax=ax)
    ax.set_xlabel(target_col, fontsize=12)
    ax.set_ylabel('Composite Score', fontsize=12)
    ax.set_title(f"Composite Score vs {target_col}\nSpearman's ρ = {r:.2f} (p={p:.2g})", fontsize=14)
    plt.tight_layout()
    # Save plot
    output_dir = "spearman's plots"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"composite_score_vs_{target_col}.png")
    plt.savefig(output_path, dpi=200)
    plt.close()
    print(f"Composite score plot saved as {output_path}")


def plot_composite_score_main():
    plot_composite_score('telomere_analysis.csv', target_col='Age')

def plot_mutational_signatures_main():
    plot_mutational_signatures('telomere_analysis.csv')
    print("Mutational signature plots saved in 'plots/' directory")

def plot_spearman_with_age_main():
    plot_spearman_with_age('telomere_analysis.csv')
    print("Spearman plots saved in 'spearman's plots/' directory")

def plot_mutation_r_heatmap(csv_path, target_col='Age'):
    """
    Plot a heatmap of Spearman r values between each normalized mutation column (per_1k/per1k) and the target column (e.g., Age),
    as well as the total mutation count column if present. Also plot a clustered heatmap of these values for all samples.
    """
    import scipy.stats as stats
    # Read data
    df = pd.read_csv(csv_path)
    # Only keep columns with numeric data and drop rows with missing target
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if target_col not in numeric_cols:
        print(f"No '{target_col}' column found in the data.")
        return
    # Only use columns with 'per_1k' or 'per1k' in the name, plus total mutation count if present
    mutation_cols = [col for col in numeric_cols if ('per_1k' in col or 'per1k' in col)]
    total_mut_col = None
    for col in numeric_cols:
        if 'total_mutation' in col and col not in mutation_cols:
            total_mut_col = col
            break
    if total_mut_col:
        mutation_cols.append(total_mut_col)
    if not mutation_cols:
        print("No normalized 'per_1k' or 'per1k' mutation columns found.")
        return
    df = df.dropna(subset=[target_col])
    # Calculate Spearman r for each mutation column
    r_values = []
    for col in mutation_cols:
        sub_df = df.dropna(subset=[col])
        if sub_df.shape[0] < 2:
            r = float('nan')
        else:
            r, _ = stats.spearmanr(sub_df[col], sub_df[target_col])
        r_values.append(r)
    # Create DataFrame for heatmap
    r_df = pd.DataFrame({'Mutation': mutation_cols, 'Spearman_r': r_values})
    r_df = r_df.set_index('Mutation')
    # Plot heatmap of r values
    plt.figure(figsize=(max(8, len(mutation_cols) * 0.4), 2.5))
    sns.heatmap(r_df.T, annot=True, cmap='coolwarm', center=0, cbar_kws={'label': "Spearman's r"})
    plt.title(f"Spearman r values: Normalized Mutations vs {target_col}")
    plt.yticks(rotation=0)
    plt.tight_layout()
    output_dir = "spearman's plots"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"mutation_r_heatmap_vs_{target_col}.png")
    plt.savefig(output_path, dpi=200)
    plt.close()
    print(f"Mutation r heatmap saved as {output_path}")

    # Clustered heatmap of normalized mutation values for all samples
    # (Removed as per user request)
    # if len(mutation_cols) > 1:
    #     import warnings
    #     with warnings.catch_warnings():
    #         warnings.simplefilter("ignore")
    #         sns.clustermap(df[mutation_cols], cmap='viridis', standard_scale=1, yticklabels=False)
    #     plt.suptitle('Clustered Heatmap of Normalized Mutation Values (Z-score by column)', y=1.02)
    #     plt.tight_layout()
    #     output_path2 = os.path.join(output_dir, f"mutation_value_clustermap.png")
    #     plt.savefig(output_path2, dpi=200)
    #     plt.close()
    #     print(f"Clustered mutation value heatmap saved as {output_path2}")


def plot_mutation_r_heatmap_main():
    plot_mutation_r_heatmap('telomere_analysis.csv', target_col='Age')

def plot_pairwise_r_heatmap(csv_path):
    """
    Plot a heatmap of pairwise Spearman r values between all relevant columns (per_1k/per1k, total mutation count, Age, and telomere columns).
    The diagonal is masked (crossed out).
    """
    import scipy.stats as stats
    # Read data
    df = pd.read_csv(csv_path)
    # Only use columns with 'per_1k' or 'per1k' in the name, plus total mutation count if present
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    mutation_cols = [col for col in numeric_cols if ('per_1k' in col or 'per1k' in col)]
    total_mut_col = None
    for col in numeric_cols:
        if 'total_mutation' in col and col not in mutation_cols:
            total_mut_col = col
            break
    if total_mut_col:
        mutation_cols.append(total_mut_col)
    # Add Age if present
    if 'Age' in df.columns and 'Age' in numeric_cols and 'Age' not in mutation_cols:
        mutation_cols.append('Age')
    # Add telomere columns (case-insensitive)
    telomere_cols = [col for col in df.columns if 'telomere' in col.lower() and col in numeric_cols and col not in mutation_cols]
    mutation_cols.extend(telomere_cols)
    if not mutation_cols:
        print("No relevant columns found for pairwise heatmap.")
        return
    # Compute pairwise Spearman r matrix
    n = len(mutation_cols)
    r_matrix = np.zeros((n, n))
    for i, col1 in enumerate(mutation_cols):
        for j, col2 in enumerate(mutation_cols):
            # Drop rows with missing values in either column
            sub_df = df[[col1, col2]].dropna()
            if sub_df.shape[0] < 2:
                r = np.nan
            else:
                r_val = stats.spearmanr(sub_df[col1], sub_df[col2])[0]
                # Ensure r_val is a scalar float
                if isinstance(r_val, numbers.Number) and not isinstance(r_val, (list, tuple, np.ndarray)):
                    r = float(r_val)
                else:
                    r = np.nan
            r_matrix[i, j] = r
    # Mask the diagonal
    mask = np.eye(n, dtype=bool)
    # Plot heatmap with improved readability
    fig_width = max(10, n * 0.7)
    fig_height = max(8, n * 0.7)
    plt.figure(figsize=(fig_width, fig_height))
    ax = sns.heatmap(
        r_matrix,
        annot=True,
        fmt=".2f",
        cmap="coolwarm",
        center=0,
        mask=mask,
        xticklabels=mutation_cols,
        yticklabels=mutation_cols,
        cbar_kws={'label': "Spearman's r"},
        annot_kws={"size": 8}
    )
    # Cross out the diagonal
    for i in range(n):
        ax.add_patch(plt.Rectangle((i, i), 1, 1, fill=False, edgecolor='black', lw=2, hatch='xx'))
    plt.title("Pairwise Spearman r Heatmap (Normalized Mutations, Age, Telomere)", fontsize=14)
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.yticks(fontsize=9)
    plt.tight_layout()
    output_dir = "spearman's plots"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "pairwise_mutation_r_heatmap.png")
    plt.savefig(output_path, dpi=200)
    plt.close()
    print(f"Pairwise mutation r heatmap saved as {output_path}")

def plot_pairwise_r_heatmap_main():
    plot_pairwise_r_heatmap('telomere_analysis.csv')

if __name__ == "__main__":
    plot_mutational_signatures_main()
    plot_spearman_with_age_main()
    plot_composite_score_main()
    plot_mutation_r_heatmap_main()
    plot_pairwise_r_heatmap_main()

### Main Function

In [3]:
import sys

# Step 1: Generate CSV
print("[1/4] Generating telomere_analysis.csv ...")
data_dir = "/Users/akhilpeddikuppa/FieldLab/GreiderCodeSearch/greider_data_download"
generate_csv(data_dir)

# Step 2: Plot histograms
print("[2/4] Plotting histograms ...")
plot_histograms_main()

# Step 3: Plot mutational signatures
print("[3/4] Plotting mutational signatures ...")
# from plotting import plot_mutational_signatures_main
# from plotting import plot_spearman_with_age_main
# from plotting import plot_composite_score_main
# from plotting import plot_mutation_r_heatmap_main
# from plotting import plot_pairwise_r_heatmap_main
plot_mutational_signatures_main()
plot_spearman_with_age_main()
plot_composite_score_main()
# plot_mutation_r_heatmap_main()
plot_pairwise_r_heatmap_main()

# Step 4: Plot trendlines and correlations
# print("[4/4] Plotting trendlines and correlations ...")
# from trendline import plot_trendlines_main
# plot_trendlines_main()

print("\nAll processing complete.")

[1/4] Generating telomere_analysis.csv ...

Processing HG002.1.NB65uq.fastq:
Age: 
Telomere Length: 
2x cstrand total: 70842
2x g strand total: 585237

Processing HG002.1.NB72uq.fastq:
Age: 
Telomere Length: 
2x cstrand total: 11649
2x g strand total: 167940

Processing HG002.1A14.NB50uq.fastq.gz:
Age: 
Telomere Length: 
2x cstrand total: 12501
2x g strand total: 112743

Processing HG002.1A14.NB68uq.fastq.gz:
Age: 
Telomere Length: 
2x cstrand total: 9898
2x g strand total: 49458

Processing HG002.1A14.NB88uq.fastq.gz:
Age: 
Telomere Length: 
2x cstrand total: 4192
2x g strand total: 18536

Processing HG002.1A15.NB68uq.fastq.gz:
Age: 
Telomere Length: 
2x cstrand total: 31869
2x g strand total: 509239

Processing HG002.1A16.NB68uq.fastq.gz:
Age: 
Telomere Length: 
2x cstrand total: 97152
2x g strand total: 865556

Processing HG002.1A16.NB88uq.fastq.gz:
Age: 
Telomere Length: 
2x cstrand total: 42789
2x g strand total: 344531

Processing HG002.1A18.NB68uq.fastq.gz:
Age: 
Telomere Length

/var/folders/xn/24qydyx54l1bvwlcysq9mzwm0000gn/T/ipykernel_45237/1873597982.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plot_data['Age_Group'] = pd.cut(plot_data['Age'], bins=age_bins, labels=age_labels, include_lowest=True)
/var/folders/xn/24qydyx54l1bvwlcysq9mzwm0000gn/T/ipykernel_45237/1873597982.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plot_data['Age_Group'] = pd.cut(plot_data['Age'], bins=age_bins, labels=age_labels, include_lowest=True)
/var/folders/xn/24qydyx54l1bvwlcysq9mzwm0000

Histogram plot saved as 'histogram.png'
[3/4] Plotting mutational signatures ...


NameError: name 'plot_mutational_signatures_main' is not defined